In [5]:
import os
import sys

import xarray as xr
import numpy as np
import cv2
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt

# Get the current working directory
CURRENT_DIR = os.getcwd()

# Set the root directory to the parent of the current directory
ROOT = os.path.abspath(os.path.join(CURRENT_DIR, os.pardir))
sys.path.insert(0, ROOT)

from code_.tools.loading import load_nsd_images
from code_.encoding_score.benchmarks.nsd import filter_activations, load_nsd_data
from code_.model_activations.models.utils import load_full_identifier
from dotenv import load_dotenv

load_dotenv()
CACHE = os.getenv("CACHE")


# tSNE

In [7]:
# Compute the coordinates of the image on the plot
def scale_to_01_range(x):
    # compute the distribution range
    value_range = (np.max(x) - np.min(x))
    # move the distribution so that it starts from zero
    # by extracting the minimal value from all its values
    starts_from_zero = x - np.min(x)

    # make the distribution fit [0; 1] by dividing by its range
    return starts_from_zero / value_range


def compute_plot_coordinates(image, x, y, image_centers_area_size, offset):
    image_height, image_width, _ = image.shape
    # compute the image center coordinates on the plot
    center_x = int(image_centers_area_size * x) + offset

    # in matplotlib, the y axis is directed upward
    # to have the same here, we need to mirror the y coordinate
    center_y = int(image_centers_area_size * (1 - y)) + offset

    # knowing the image center,
    # compute the coordinates of the top left and bottom right corner
    tl_x = center_x - int(image_width / 2)
    tl_y = center_y - int(image_height / 2)

    br_x = tl_x + image_width
    br_y = tl_y + image_height

    return tl_x, tl_y, br_x, br_y
    

def scale_image(image, max_image_size):
    image_height, image_width, _ = image.shape
    scale = max(1, image_width / max_image_size, image_height / max_image_size)
    image_width = int(image_width / scale)
    image_height = int(image_height / scale)

    image = cv2.resize(image, (image_width, image_height))
    return image


def draw_rectangle_by_class(image, label):
    image_height, image_width, _ = image.shape
    # get the color corresponding to image class
    color = colors_per_class[label]
    image = cv2.rectangle(image, (0, 0), (image_width - 1, image_height - 1), color=color, thickness=5)

    return image


def visualize_tsne_images(tx, ty, images, name, dpi = 300, plot_size=1000, max_image_size=100):
    # we'll put the image centers in the central area of the plot
    # and use offsets to make sure the images fit the plot
    offset = max_image_size // 2
    image_centers_area_size = plot_size - 2 * offset

    tsne_plot = 255 * np.ones((plot_size, plot_size, 3), np.uint8)

    # now we'll put a small copy of every image to its corresponding T-SNE coordinate
    for i in range(len(images)):
    
        x = tx[i]
        y = ty[i]
        
        try:
            image = cv2.imread(images[i])
            # scale the image to put it to the plot
            image = scale_image(image, max_image_size)

            # draw a rectangle with a color corresponding to the image class
            #image = draw_rectangle_by_class(image, label)

            # compute the coordinates of the image on the scaled plot visualization
            tl_x, tl_y, br_x, br_y = compute_plot_coordinates(image, x, y, image_centers_area_size, offset)

            # put the image to its TSNE coordinates using numpy subarray indices
            tsne_plot[tl_y:br_y, tl_x:br_x, :] = image
        
        
        except AttributeError:
            pass


    fig = plt.figure(figsize=(80,50),dpi=100)
    plt.imshow(tsne_plot[:, :, ::-1])
    plt.savefig(os.path.join(ROOT,f'figures/figure_4.png'), dpi=dpi) 


In [8]:
# load model activations
identifier = load_full_identifier(model_name='expansion', features=3000, layers=5, dataset='naturalscenes')
features = xr.open_dataarray(os.path.join(CACHE,'activations',identifier),engine='netcdf4')

# load images seen by subject 1
subject = 1
IDS = load_nsd_data(mode='unshared', subject=subject, region='V1')[0]
images  = [image for image in load_nsd_images() if os.path.basename(image).strip('.png') in IDS]

# filter activations for images seen by subject 1
features = filter_activations(features, IDS)
tsne_ints = TSNE(n_components=2).fit_transform(features)
tx = tsne_ints[:, 0]
ty = tsne_ints[:, 1]
tx = scale_to_01_range(tx)
ty = scale_to_01_range(ty)
dpi  = 800
# visualize 
visualize_tsne_images(tx, ty, images, 
                      dpi = dpi, 
                      plot_size=1000, 
                      max_image_size=10)

FileNotFoundError: [Errno 2] No such file or directory: '/home/atlask/data/atlas/.cache/activations/expansion_features=3000_layers=5_dataset=naturalscenes'